## Call the Abbys

In [5]:
# Import necessary libraries
import numpy as np 
import pandas as pd
import re
import sqlite3
import pickle  # To save the model

# Scikit-learn modules for machine learning pipeline and model tuning
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, KFold
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import LabelEncoder

# Scikit-learn modules for feature extraction (text vectorization)
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

# Machine learning algorithms
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC

# Scikit-learn metrics for model evaluation
from sklearn.metrics import f1_score, precision_score, recall_score, accuracy_score, classification_report, confusion_matrix


In [6]:
# Establish a connection to the SQLite database
db_path = r'E:\Sentimen\sentimen\data.db'
connection = sqlite3.connect(db_path, check_same_thread=False)

# Define SQL query to retrieve data from 'a_train' table
sql_query = 'SELECT * FROM a_trainb'

# Execute the query and load data into a DataFrame
a_train_data = pd.read_sql(sql_query, connection)

# Display the first few rows of the retrieved data
a_train_data.head()


,Teks,Label
0,warung dimiliki pengusaha pabrik tahu puluhan ...,positive
1,mohon ulama lurus mmbri hujjah partai diwlh su...,neutral
2,lokasi strategis jalan sumatra bandung nyaman ...,positive
3,betapa bahagia unboxing paket barang bagus men...,positive
4,aduh mahasiswa jangan sombong dong kasih kartu...,negative


In [7]:
# Menghitung jumlah kemunculan masing-masing nilai di kolom 'label'
label_counts = a_train_data['Label'].value_counts()

# Menampilkan nilai frekuensi masing-masing kategori
print(label_counts)

# Menghitung total jumlah data
total_count = label_counts.sum()

# Menampilkan total jumlah data
print(f"Total count: {total_count}")

Label
positive    6416
negative    3436
neutral     1148
Name: count, dtype: int64
Total count: 11000


## Encoding

In [8]:
data_preprocessed = a_train_data.Teks.tolist()

In [ ]:
data_preprocessed

In [10]:
# Untuk melakukan Feature Extraction, kita menggunakan library "Sklearn atau scikit-learn".
# Sklearn adalah library untuk melakukan task-task Machine Learning.
# "CountVectorizer" merupakan salah satu modul untuk melakukan "BoW"

from sklearn.feature_extraction.text import CountVectorizer

# Kita proses Feature Extraction
count_vect = CountVectorizer()
count_vect.fit(data_preprocessed)

X = count_vect.transform(data_preprocessed)
print ("Feature Extraction selesai")

Feature Extraction selesai


In [11]:
X

<Compressed Sparse Row sparse matrix of dtype 'int64'
	with 167026 stored elements and shape (11000, 15320)>

In [8]:
import pickle

pickle.dump(count_vect, open("feature.pkl", "wb"))

In [14]:
from sklearn.model_selection import train_test_split

classes = a_train_data.Label

In [15]:
classes

0        positive
1         neutral
2        positive
3        positive
4        negative
           ...   
10995    positive
10996    positive
10997     neutral
10998    negative
10999    positive
Name: Label, Length: 11000, dtype: object

melakukan encoding label kategori ke dalam format one-hot encoding agar dapat digunakan dalam model Machine Learning.
Data teks dan label yang telah di-encode dipisahkan menjadi fitur (x) dan target (y).

Melakukan one-hot encoding pada label kategori penting untuk model machine learning karena model ini umumnya tidak bisa langsung memproses data kategorikal. Encoding ini mengubah label menjadi format biner yang dapat dipahami secara numerik, menghindari misinterpretasi urutan, dan menghilangkan bias dalam model. Dengan membagi data teks dan label yang telah di-encode menjadi fitur (X) dan target (Y), kita memastikan bahwa model dapat melatih dan memprediksi dengan benar menggunakan fungsi aktivasi dan loss function yang sesuai

## Train Test Split x train, x test, y train, y test

membagi data menjadi dua subset: data pelatihan (X_train, y_train) dan data pengujian (X_test, y_test). Dengan test_size=0.2, 20% dari data digunakan untuk pengujian dan 80% untuk pelatihan

In [16]:
X_train, X_test, y_train, y_test = train_test_split(X, classes, test_size = 0.2)

Multilayer Perceptron

In [17]:
from sklearn.neural_network import MLPClassifier

model = MLPClassifier() 
model.fit(X_train, y_train)

print ("Training selesai")

Training selesai


In [18]:
pickle.dump(model, open("model.pkl", "wb"))

model tampaknya lebih efektif dalam mengklasifikasikan kelas positive dan negative, sementara performanya sedikit kurang untuk kelas neutral. Akurasi makro rata-rata adalah 0.77, menunjukkan performa yang konsisten di berbagai kelas.

In [19]:
from sklearn.metrics import classification_report

test = model.predict(X_test)

print ("Testing Ok")

print(classification_report(y_test, test)) 

Testing Ok
              precision    recall  f1-score   support

    negative       0.77      0.79      0.78       675
     neutral       0.79      0.67      0.73       243
    positive       0.88      0.89      0.89      1282

    accuracy                           0.84      2200
   macro avg       0.81      0.79      0.80      2200
weighted avg       0.84      0.84      0.84      2200



## Cross Validation

cross-validation dengan KFold untuk mengevaluasi model. Data dibagi menjadi 5 lipatan

In [20]:
# "Cross Validation"
import numpy as np
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold

kf = KFold(n_splits=5,random_state=42,shuffle=True)

accuracies = []

y = classes

for iteration, data in enumerate(kf.split(X), start=1):

    data_train   = X[data[0]]
    target_train = y[data[0]]

    data_test    = X[data[1]]
    target_test  = y[data[1]]

    clf = MLPClassifier()
    clf.fit(data_train,target_train)

    preds = clf.predict(data_test)

    # for the current fold only    
    accuracy = accuracy_score(target_test,preds)

    print("Iterasi ke-", iteration)
    print(classification_report(target_test,preds))
    print("="*20)

    accuracies.append(accuracy)

# this is the average accuracy over all folds
average_accuracy = np.mean(accuracies)

print()
print()
print()
print("Rata-rata Accuracy: ", average_accuracy)

Iterasi ke- 1
              precision    recall  f1-score   support

    negative       0.76      0.78      0.77       680
     neutral       0.77      0.60      0.67       239
    positive       0.87      0.89      0.88      1281

    accuracy                           0.82      2200
   macro avg       0.80      0.76      0.77      2200
weighted avg       0.82      0.82      0.82      2200

Iterasi ke- 2
              precision    recall  f1-score   support

    negative       0.78      0.75      0.76       706
     neutral       0.66      0.65      0.66       220
    positive       0.87      0.89      0.88      1274

    accuracy                           0.82      2200
   macro avg       0.77      0.77      0.77      2200
weighted avg       0.82      0.82      0.82      2200

Iterasi ke- 3
              precision    recall  f1-score   support

    negative       0.78      0.76      0.77       682
     neutral       0.78      0.67      0.72       215
    positive       0.87      0.90

Secara keseluruhan, model menunjukkan performa yang baik dalam mengklasifikasikan kelas positive dan negative, tetapi perlu diperhatikan bahwa hasil untuk kelas neutral lebih bervariasi dan cenderung lebih rendah. Rata-rata akurasi yang stabil menunjukkan bahwa model tidak terlalu overfit pada data latih dan generalisasinya baik pada data uji.

## Predict Model

clean_text(text): Membersihkan teks dengan mengubahnya menjadi huruf kecil, menghapus URL, karakter non-alfabet, dan spasi berlebih.

standardize_text(text): Mengganti kata-kata "alay" dengan bentuk baku menggunakan kamus dari database.

remove_stopwords(text): Menghapus kata-kata berhenti dari teks jika diperlukan (fungsi ini saat ini dikomentari).

preprocess_input(text): Menggabungkan pembersihan teks dan standarisasi teks, serta menghapus stopwords jika diaktifkan.

Feature Extraction dan Prediksi: Mengubah teks yang diproses menjadi fitur yang bisa dipahami oleh model dan menggunakan model untuk memprediksi sentimen teks.

In [29]:
import re
import pandas as pd
import numpy as np
import sqlite3
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split

# Menghubungkan ke database SQLite
db_path = r'E:\Sentimen\sentimen\data.db'
connection = sqlite3.connect(db_path)

def clean_text(text):
    text = text.lower()  # Ubah teks menjadi huruf kecil
    text = re.sub(r'((www\.[^\s]+)|(https?://[^\s]+)|(http?://[^\s]+))', '', text)  # Hapus URL dan tautan
    text = re.sub(r'pic.twitter.com\.\w+', '', text)
    text = re.sub(r'[^a-z\s]', ' ', text)  # Hapus karakter yang tidak diinginkan, termasuk angka
    text = text.replace('user', '')  # Hapus kata 'user'
    text = re.sub(' +', ' ', text)  # Hapus spasi berlebih
    text = text.replace('\n', ' ')  # Hapus karakter \n (newline)
    return text.strip()

def standardize_text(text):
    alay_df = pd.read_sql_query('SELECT * FROM kamus_alay', connection)
    alay_dict = dict(zip(alay_df['alay'], alay_df['fix']))  # Buat kamus dari teks alay ke baku
    words = text.split()
    
    standardized_words = [alay_dict.get(word, word) for word in words]  # Ganti teks alay dengan baku
    return ' '.join(standardized_words).strip()

def remove_stopwords(text):
    stopword_query = 'SELECT * FROM stopword'
    stopword_df = pd.read_sql_query(stopword_query, connection)
    stopwords = set(stopword_df['stop'])
    words = text.split()
    filtered_words = [word for word in words if word not in stopwords]
    return ' '.join(filtered_words)

def preprocess_input(text):
    text = clean_text(text)
    text = standardize_text(text)
    # text = remove_stopwords(text)  # Uncomment jika perlu menghapus stopword
    return text

# Load model and CountVectorizer from files
with open('feature.pkl', 'rb') as f:
    count_vect = pickle.load(f)
    
with open('model.pkl', 'rb') as f:
    model = pickle.load(f)

# Original text
original_text = 'Dia mah apa si jarwo, udah jelek item dan bau bawang'

# Preprocess the input text
preprocessed_text = preprocess_input(original_text)

# Feature Extraction
text_features = count_vect.transform([preprocessed_text])

# Display feature extraction
print("Input Text:")
print(original_text)
print("\nText Preprocessing:")
print(preprocessed_text)
print("\nFeature Extraction:")
print(text_features)

# Predict sentiment
result = model.predict(text_features)[0]
print("\nSentiment:")
print(result)


Input Text:
Dia mah apa si jarwo, udah jelek item dan bau bawang

Text Preprocessing:
dia adalah apa sih jarwo sudah jelek hitam dan bau bawang

Feature Extraction:
<Compressed Sparse Row sparse matrix of dtype 'int64'
	with 5 stored elements and shape (1, 15320)>
  Coords	Values
  (0, 949)	1
  (0, 957)	1
  (0, 5210)	1
  (0, 5773)	1
  (0, 12961)	1

Sentiment:
negative


In [23]:
load_model = pickle.load(open('feature.pkl', 'rb'))
model_NN=pickle.load(open('model.pkl', 'rb'))

In [4]:
import re
import pandas as pd
import sqlite3
from sklearn.preprocessing import StandardScaler

# Menghubungkan ke database SQLite
db_path = r'E:\Sentimen\sentimen\data.db'
connection = sqlite3.connect(db_path)

# Fungsi membersihkan teks
def clean_text(text):
    text = re.sub(r'((www\.[^\s]+)|(https?://[^\s]+)|(http?://[^\s]+))', '', text)  # Hapus URL dan tautan
    text = re.sub(r'pic.twitter.com\.\w+', '', text)
    text = re.sub(r'[^a-z\s]', ' ', text.lower())  # Hapus karakter yang tidak diinginkan, termasuk angka
    text = text.replace('user', '')  # Hapus kata 'user'
    text = re.sub(' +', ' ', text)  # Hapus spasi berlebih
    text = text.replace('\n', ' ')  # Hapus karakter \n (newline)
    return text.strip()

# Fungsi standarisasi teks (ganti teks alay dengan baku)
def standardize_text(text):
    alay_df = pd.read_sql_query('SELECT * FROM kamus_alay', connection)
    alay_dict = dict(zip(alay_df['alay'], alay_df['fix']))  # Buat kamus dari teks alay ke baku
    words = text.split()
    standardized_words = [alay_dict.get(word, word) for word in words]  # Ganti teks alay dengan baku
    return ' '.join(standardized_words).strip()

# Fungsi hapus stopwords
def remove_stopwords(text):
    stopword_query = 'SELECT * FROM stopword'
    stopword_df = pd.read_sql_query(stopword_query, connection)
    stopwords = set(stopword_df['stop'])
    words = text.split()
    filtered_words = [word for word in words if word not in stopwords]
    return ' '.join(filtered_words)

# Fungsi untuk melakukan preprocessing teks input dan mengembalikan teks asli dan yang sudah dibersihkan
def preprocess_input(text):
    original_text = text  # Simpan teks asli
    text = clean_text(text)
    text = standardize_text(text)
    # Jika ingin menghapus stopwords, uncomment ini
    # text = remove_stopwords(text)
    cleaned_text = text  # Simpan teks yang telah dibersihkan
    return original_text, cleaned_text

# Mengambil teks yang akan diprediksi
original_text = "Si Jarwo teh, emg bodo bgt"

# Preprocessing teks dan menampilkan hasilnya
original_text, cleaned_text = preprocess_input(original_text)

# Tampilkan teks asli dan teks hasil preprocessing
print(f"Teks Asli: {original_text}")
print(f"Teks Setelah Cleaning: {cleaned_text}")

# Feature Extraction (misalnya menggunakan load_model transform untuk vektorisasi)
# Misalkan 'load_model' adalah model yang digunakan untuk ekstraksi fitur teks
text = load_model.transform([cleaned_text])  # Pastikan sesuai dengan cara ekstraksi fitur yang digunakan

# Kita prediksi sentimennya
result = model_NN.predict(text)[0]

# Tampilkan hasil prediksi
print("Sentimen:")
print(result)


Teks Asli: Si Jarwo teh, emg bodo bgt
Teks Setelah Cleaning: sih jarwo teh memang bodoh banget
Sentimen:
negative
